In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

#list contents in your drive
!ls /mydrive

In [ ]:
!pip install tensorflow==1.15

In [ ]:
!pip install matplotlib

In [ ]:
!pip install opencv-python

In [ ]:
import glob
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt

def get_annotations(annot_name):
    with open("/mydrive/test_YOLO_format/" +annot_name) as f:
        lines = f.readlines()
        annot = []
        for line in lines:
            l_arr = line.split(" ")[1:5]
            l_arr = [int(i) for i in l_arr]
            annot.append(l_arr)
    return annot

for im_name in sorted(glob.glob('/mydrive/test/*.png')):
    nam = im_name
    h = nam.replace("/mydrive/test/", "")
    k = h.replace(".png", "")
    l = int(k)
    print(l)
    if l > 200:
      # Read the graph.
      with tf.gfile.FastGFile('/mydrive/frozen_inference_graph14700.pb', 'rb') as f:
          graph_def = tf.GraphDef()
          graph_def.ParseFromString(f.read())

      with tf.Session() as sess:
          # Restore session
          sess.graph.as_default()
          tf.import_graph_def(graph_def, name='')

          # Read and preprocess an image.
          img = cv.imread(im_name)
          im_width = img.shape[0]
          im_height = img.shape[1]
          inp = cv.resize(img, (300, 300))
          inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

          # Run the model
          out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                          sess.graph.get_tensor_by_name('detection_scores:0'),
                          sess.graph.get_tensor_by_name('detection_boxes:0'),
                          sess.graph.get_tensor_by_name('detection_classes:0')],
                        feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

          # Visualize detected bounding boxes.
          num_detections = int(out[0][0])

          class_ids = []
          confidences = []
          boxes = []
          for i in range(num_detections):
              classId = int(out[3][0][i])
              score = float(out[1][0][i])
              bbox = [float(v) for v in out[2][0][i]]
              if score > 0.3:
                  xmin, ymin, xmax, ymax = bbox
                  (xmin, xmax, ymin, ymax) = (xmin * im_width, xmax * im_width,
                                              ymin * im_height, ymax * im_height)
                  boxes.append([int(ymin), int(xmin), int(ymax-ymin), int(xmax-xmin)])
                  confidences.append(float(score))
                  class_ids.append(0)
          print(boxes)
          name = im_name.replace("/mydrive/test/", "/mydrive/annotations/")
          annot_name = name.replace(".png", ".txt")
          print(annot_name)
          f = open(annot_name, "w")
          f.write(str(boxes))
          f.close()
 


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
Instructions for updating:
Use tf.gfile.GFile.
[[286, 116, 26, 67]]
/mydrive/annotations/0201.txt
202
[]
/mydrive/annotations/0202.txt
203
[[19, 163, 65, 83]]
/mydrive/annotations/0203.txt
204
[[142, 145, 30, 58]]
/mydrive/annotations/0204.txt
205
[[114, 85, 32, 46]]
/mydrive/annotations/0205.txt
206
[[